In [ ]:
import numpy as np
import torch 
from matplotlib import pyplot as plt
from utils import radonTransform,initial
from skimage.transform import rescale
from skimage.data import shepp_logan_phantom
from MCMC.BPS_Gibbs_sparse import BPS_Gibbs
from MCMC.GS_Horseshoe import Gibbs_sampling
from MCMC.PLD_sparse import proximal_langevin
from MCMC.PCN_KL_sparse import PCN


In [ ]:
pixel=64
np.random.seed(5041294)
height, width = 64,64
angleNum = 32
A = radonTransform(angleNum, pixel, pixel)
image = shepp_logan_phantom()
x_true = rescale(image, scale=height / 400, mode='reflect')

x = x_true.reshape(-1, 1)
y_noise_free = A @ x
sigma = 0.01 * np.max(y_noise_free)
y = (y_noise_free + sigma * np.random.randn(*y_noise_free.shape))
n = height * width
m = y.shape[0]


In [ ]:
plt.figure()
plt.imshow(x.reshape(height, width))
plt.colorbar(plt.imshow(x.reshape(height, width)))
plt.show()

In [ ]:
device1 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
x_init=initial(A,y,device1)

In [ ]:
hyper=[10,1,10,1,1,1]


In [ ]:
x_mean,x_var = BPS_Gibbs(x_init,torch.from_numpy(y).to(device1),torch.from_numpy(A.copy()).to(device1),sigma,hyper)

In [ ]:
pixel=64
plt.figure()
plt.imshow((x_mean.view(pixel,pixel)).cpu().numpy())
plt.colorbar(plt.imshow((x_mean.view(pixel,pixel)).cpu().numpy()))
plt.show()